In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp
import GPy

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

In [ ]:
# make deterministic
np.random.seed(100)

Function to optimize:

In [ ]:
xmin, xmax = -6, 6
ymin, ymax = -5, 5
x = np.linspace(xmin, xmax, 100)
y = np.linspace(ymin, ymax, 100)
f = lambda x,y: 1.5 * (np.sin(0.5*x)**2 * np.cos(y) + 0.1*x + 0.2*y) + \
    np.random.normal(0, 0.2, size=None if isinstance(x, float) else x.shape)
X, Y = np.meshgrid(x, y)
Z = f(X,Y)
best_z = np.min(Z)

In [ ]:
tp.surface_3D(X,Y,Z)

In [ ]:
bounds = [('x', xmin, xmax), ('y', ymin, ymax)]

op = tb.Optimiser(f, 'min', bounds, pre_phase_trials=8, settings_preset='default')
beta = lambda trial_num: np.log(trial_num**2)/2-1
op.acq_func_factory = tm.UCB.Factory(beta=beta)
kernel = GPy.kern.RBF(input_dim=2)
#kernel.variance.set_prior(GPy.priors.Gamma.from_EV(8, 6))
#kernel.lengthscale.set_prior(GPy.priors.Gamma.from_EV(1, 2))
op.surrogate = tm.GPySurrogate(training_iterations=10)
'''
op.surrogate_factory = tm.SciKitGPSurrogate.Factory(gp_params=dict(
    kernel = gp.kernels.ConstantKernel(constant_value_bounds=(0.1, 5)) * gp.kernels.RBF(length_scale_bounds=(0.1, 5)) + gp.kernels.WhiteKernel(noise_level_bounds=(1e-05, 0.5)),
    #kernel = gp.kernels.ConstantKernel() * gp.kernels.RBF() + gp.kernels.WhiteKernel(),
), variable_iterations=lambda trial_num: [10,5,2][(trial_num-4) % 3])
'''
rec = tp.PlottingRecorder(op)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=30)

In [ ]:
ts = np.array(range(op.pre_phase_trials, len(rec.trials)))
plt.plot(ts, beta(ts), label='beta')
plt.legend()
plt.show()

In [ ]:
tp.plot_error_over_time(rec, true_best=best_z);

In [ ]:
op.get_incumbent()

In [ ]:
for n,t in rec.trials.items():
    model = t.selection_info.get('model')
    if model is not None:
        hyper_params = model.get_hyper_params()
        print('{:02d}: params: {}'.format(n, hyper_params))

In [ ]:
tp.plot_surrogate_hyper_params_2D(rec, param_indices=(0,1), size_limits=(30,30), use_param_bounds=True);

In [ ]:
tp.interactive_plot_trial_1D(rec);

In [ ]:
tp.interactive_plot_trial_2D(rec, true_objective=f)

# Try optimising the same function with random search

In [ ]:
ra = tb.Optimiser(f, 'min', bounds, pre_phase_trials=1, settings_preset='random_search')
recr = tp.PlottingRecorder(ra)
ra.run(max_trials=1000)

In [ ]:
tp.plot_error_over_time(recr, true_best=best_z)

In [ ]:
ra.get_incumbent()

In [ ]:
tp.interactive_plot_trial_2D(recr, true_objective=f)